In [ ]:
#подвязка кегля
!rm -rf /root/.kaggle/
!mkdir /root/.kaggle
!touch /root/.kaggle/kaggle.json

!echo '{"username":"***","key":"***"}' >> /root/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d andrewmvd/trip-advisor-hotel-reviews
!unzip -q trip-advisor-hotel-reviews.zip

  0% 0.00/5.14M [00:00<?, ?B/s]
100% 5.14M/5.14M [00:00<00:00, 64.7MB/s]


In [ ]:
#установка
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark


In [ ]:
#установка переменных окружения
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

#создание спарк сессии
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
data = spark.read.csv(
    'tripadvisor_hotel_reviews.csv',
        inferSchema=True, header=True, )


In [ ]:
data.show()


+--------------------+------+
|              Review|Rating|
+--------------------+------+
|nice hotel expens...|     4|
|ok nothing specia...|     2|
|nice rooms not 4*...|     3|
|unique, great sta...|     5|
|great stay great ...|     5|
|love monaco staff...|     5|
|cozy stay rainy c...|     5|
|excellent staff, ...|     4|
|hotel stayed hote...|     5|
|excellent stayed ...|     5|
|poor value stayed...|     2|
|nice value seattl...|     4|
|nice hotel good l...|     4|
|nice hotel not ni...|     3|
|great hotel night...|     4|
|horrible customer...|     1|
|disappointed say ...|     2|
|fantastic stay mo...|     5|
|good choice hotel...|     5|
|hmmmmm say really...|     3|
+--------------------+------+
only showing top 20 rows



In [ ]:
from pyspark.sql import functions as F

data_lower = data.select(F.lower(F.col('Review')).alias('Review'), 'Rating') # приведены к нижнему регистру
data_lower = data_lower.select(F.regexp_replace(F.col("Review"), " +", " ").alias('Review'), 'Rating') # избавляемся от двойных пробелов
data_lower_woSCh = data_lower.select(F.regexp_replace(F.col("Review"), "[^a-z\s.]", "").alias('Review'), 'Rating') # удалены спец символы
data_lower_woSCh = data_lower_woSCh.select(F.regexp_replace(F.col("Review"), " +", " ").alias('Review'), 'Rating')

In [ ]:
from pyspark.sql.types import *
import collections
import nltk
from nltk.probability import FreqDist

dfSchema = schema = StructType([
    StructField('Number of review', LongType(), True),
    StructField('Number of sentence', LongType(), True),
    StructField('Word', StringType(), True),
    StructField('Count in sentence', LongType(), True)
])

# создаем таблицу с частотой появления слов в предложениях

listDataFrame = []
listLengthReview = []
listReview = list(data_lower_woSCh.select('Review').toPandas()['Review'])
countRows = len(listReview)
for i in range(0, countRows):
  sentences = listReview[i].split(".")
  countSentence = len(sentences)
  length = 0
  for j in range(0, countSentence):
    sentences[j] = sentences[j].strip()
    sentences[j] = sentences[j].replace('  ', ' ')
    words = sentences[j].split(" ")
    while(words.count('') > 0):
      words.remove('')
    length = length + len(words)
    text = nltk.Text(words)
    fdist = FreqDist(text)
    for key, value in fdist.items():
      listDataFrame.append([i + 1, j + 1, key, value])
  listLengthReview.append([i + 1, length])

dataWordsSentences = spark.createDataFrame(listDataFrame, schema)

dataWordsSentences.orderBy('Count in sentence', ascending = False).show(100)


+----------------+------------------+----------+-----------------+
|Number of review|Number of sentence|      Word|Count in sentence|
+----------------+------------------+----------+-----------------+
|           17160|                 1|       not|               22|
|            4114|                 2|      pool|               19|
|           17977|                 4|       not|               18|
|            2493|                 1|     hotel|               17|
|           18632|                 2|      food|               17|
|            2996|                 2|       not|               17|
|            2961|                 1|       not|               17|
|           17126|                 1|        nt|               16|
|           18632|                 1|      food|               16|
|           13004|                 3|        nt|               16|
|           20127|                 1|        nt|               16|
|            4116|                 1|     hotel|              

In [ ]:
dfSchema = schema = StructType([
    StructField('Number of review', LongType(), True),
    StructField('Word', StringType(), True),
    StructField('Count in review', LongType(), True)
])

# создаем таблицу с частотой появления слов в Review

listDataFrame = []
listReview = list(data_lower_woSCh.select('Review').toPandas()['Review'])
countRows = len(listReview)
for i in range(0, countRows):
  listReview[i] = listReview[i].strip()
  review = listReview[i].replace('.', '')
  review = review.replace('  ', ' ')
  countReview = len(review)
  words = review.split(" ")
  while(words.count('') > 0):
      words.remove('')
  text = nltk.Text(words)
  fdist = FreqDist(text)
  for key, value in fdist.items():
    listDataFrame.append([i + 1, key, value])

dataWordsReview = spark.createDataFrame(listDataFrame, schema)

dataWordsReview.orderBy('Count in review', ascending = False).show(100)

+----------------+-------+---------------+
|Number of review|   Word|Count in review|
+----------------+-------+---------------+
|           17778|     nt|             52|
|           17778|    did|             36|
|           20112|  hotel|             34|
|           20490|  hotel|             33|
|           18632|   food|             32|
|            2981|     nt|             31|
|           18594|     nt|             30|
|           20112|    not|             29|
|           13004|     nt|             29|
|           18693|    not|             29|
|           14117|    yes|             27|
|           18944|     nt|             27|
|           14117|   just|             27|
|            6629|    not|             27|
|            6822|    did|             27|
|           17213|     nt|             27|
|           18918|    not|             27|
|            7073|   room|             26|
|           20112|    did|             26|
|            1798|   room|             26|
|          

In [ ]:
from pyspark import SparkContext as sc
from pyspark.sql import Row

frequencyDataFrame = dataWordsSentences.join(dataWordsReview, ['Word', 'Number of review'] )

dataLength = spark.createDataFrame(listLengthReview, ['Number of review','Length of review'])

tfidf = frequencyDataFrame.join(dataLength, ['Number of review'])

# считаем TF

tfidf = tfidf.withColumn('TF', F.col('Count in review') /  F.col('Length of review'))

# считаем IDF

idf = frequencyDataFrame.groupBy('Word').count()

tfidf = tfidf.join(idf, 'Word')

lengthDF = tfidf.count()

help = tfidf.withColumn('quo', lengthDF / F.col('count'))

help = help.select(F.log(10.0, F.col('quo')).alias('IDF'), 'Word', 'Number of review')

tfidf = tfidf.join(help, ['Word', 'Number of review'])

# считаем TF-IDF

tfidf = tfidf.withColumn('TF-IDF', F.col('TF') * F.col('IDF'))

tfidf = tfidf.withColumnRenamed('count', 'Count words in doc')

tfidf.orderBy('Count in sentence', ascending = False).show(100)



+-------+----------------+------------------+-----------------+---------------+----------------+--------------------+------------------+------------------+--------------------+
|   Word|Number of review|Number of sentence|Count in sentence|Count in review|Length of review|                  TF|Count words in doc|               IDF|              TF-IDF|
+-------+----------------+------------------+-----------------+---------------+----------------+--------------------+------------------+------------------+--------------------+
|    not|           17160|                 1|               22|             22|             556|0.039568345323741004|             19111|1.9718470901016945| 0.07802272658675768|
|   pool|            4114|                 2|               19|             20|             293| 0.06825938566552901|              4955| 2.558086843773366| 0.17461343643504207|
|   pool|            4114|                 2|               19|             20|             293| 0.0682593856655290

In [ ]:
dfSchema = schema = StructType([
    StructField('Word', StringType(), True),
    StructField('Count in docs', LongType(), True)
])

# создаем таблицу с частотой появления слов во всех документах

listDataFrame = []
listReview = list(data_lower_woSCh.select('Review').toPandas()['Review'])
strReview = ' '.join(listReview)
strReview = strReview.strip()
review = strReview.replace('.', '')
review = review.replace('  ', ' ')
words = review.split(" ")
while(words.count('') > 0):
    words.remove('')
text = nltk.Text(words)
fdist = FreqDist(text)
for key, value in fdist.items():
  listDataFrame.append([key, value])

dataWordsDataFrame = spark.createDataFrame(listDataFrame, schema)

dataWordsDataFrame.orderBy('Count in docs', ascending = False).show(100)

+-----------+-------------+
|       Word|Count in docs|
+-----------+-------------+
|      hotel|        48920|
|       room|        34347|
|        not|        31527|
|      great|        21097|
|         nt|        19000|
|       good|        16990|
|      staff|        16216|
|       stay|        15160|
|        did|        13896|
|       just|        12592|
|       nice|        12412|
|      rooms|        12031|
|         no|        11620|
|   location|        11045|
|     stayed|        10469|
|    service|         9980|
|       time|         9834|
|      night|         9739|
|      beach|         9597|
|        day|         9551|
|      clean|         9364|
|  breakfast|         9274|
|       food|         9010|
|       like|         8114|
|     resort|         7936|
|     really|         7765|
|      place|         7693|
|       pool|         7175|
|     people|         6766|
|   friendly|         6675|
|      small|         6528|
|     little|         6243|
|        got|       